<a href="https://colab.research.google.com/github/slz4025/twitter_latent_scams/blob/master/tweet2tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_name = "with_score_filtered_covid_20200602" #@param {type: "string"}
analyzed_dir = "drive/My Drive/latent_scams/analyzed/"#@param {type: "string"}

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

## Tokenization

Open and select the desired unit for processing.

In [ ]:
tweets_pd = pd.read_csv(analyzed_dir + file_name + ".tsv", sep="\t")

###Tokenization

In [ ]:
clean_docs = scored_tweets_pd["body"]
clean_pd = scored_tweets_pd

In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
print(stopwords.words('english'))

In [ ]:
# taking out !?#$@'
modified_punc = """
.,"&()*+-/:;<=>[\]^_`{|}~
"""

In [ ]:
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def de_emojify(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regex_pattern.sub(r'', text)

#https://help.twitter.com/en/using-twitter/replies-not-showing-up-and-hashtag-problems
# (1) words with opt. apostrophe
# (2) words with opt. @ prefix
# (3) words with opt. # prefix
pattern = r"""\w+(?:'\w+)*|@?\w+|#?\w+"""

#https://www.nltk.org/book/ch03.html#sec-tokenization
def tweet_tokenize(text):
  return nltk.regexp_tokenize(text, pattern)

def clean_body(body):
  body = body.lower()
  de_punc = body.translate(str.maketrans('', '', modified_punc))
  de_emoji = de_emojify(de_punc)
  tokens = tweet_tokenize(de_emoji) 
  de_stop = [word for word in tokens if word not in stopwords.words('english')]
  rem_url = [word for word in tokens if word[:4] != "http"]
  num_tok = ["NUM" if word.isdigit() else word for word in tokens]
  return num_tok

# test
print(clean_body("We'll see about that @missy #revenge"))
print(clean_body(".,!$'?\"&()*+-/:;<=>[]^_`{|}~#@'"))
print(clean_body("U.S.A U.S.A. u.s.a u.s.a. U.S. U."))
print(clean_body("He @bud It's a long walk home😢 #tears."))
print(clean_body("https://tco.com/3948hJKL*efoW"))
print(clean_body("I've got $12.00 and 40% people want it."))

In [ ]:
cleaned = []
for c in clean_docs:
  cleaned.append(clean_body(c))

In [ ]:
selected_prefix = "processing_all"#@param {type: "string"}
cleaned_file_name = selected_prefix + "_" + file_name + ".tsv"

In [ ]:
_cleaned = [' '.join(c) for c in cleaned]
clean_pd["cleaned"] = _cleaned
clean_pd.to_csv(sep='\t', path_or_buf=analyzed_dir + cleaned_file_name)

In [ ]:
clean_pd = pd.read_csv(analyzed_dir + cleaned_file_name, sep="\t")
_cleaned = clean_pd["cleaned"]
cleaned = [c.split(' ') for c in _cleaned]

###Phrases

In [ ]:
phrases_docs = cleaned
phrases_pd = clean_pd

In [ ]:
# phrases
from gensim.models import (
    phrases,
    Phrases,
)
bigram = Phrases(phrases_docs, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[phrases_docs], threshold=100) 

bigram_mod = phrases.Phraser(bigram)
trigram_mod = phrases.Phraser(trigram)

phrases = [trigram_mod[bigram_mod[doc]] for doc in phrases_docs]

In [ ]:
selected_prefix = "processing_all"#@param {type: "string"}
phrases_file_name = selected_prefix + "_" + file_name + ".tsv"

In [ ]:
_phrases = [' '.join(p) for p in phrases]
phrases_pd["phrases"] = _phrases
phrases_pd.to_csv(sep='\t', path_or_buf=analyzed_dir + phrases_file_name)

In [ ]:
phrases_pd = pd.read_csv(analyzed_dir + phrases_file_name, sep="\t")
_phrases = phrases_pd["phrases"]
phrases = [p.split(' ') for p in _phrases]

###Lemmatization - probably do not need
Different stems, especially regarding scams, actually to seem to have significant impact on whether the tweet is relevant or not.

In [ ]:
phrases_pd = pd.read_csv(analyzed_dir + phrases_file_name, sep="\t")
_phrases = clean_pd["phrases"]
phrases = [p.split(' ') for p in _phrases]
lemma_docs = phrases
lemma_pd = phrases_pd

In [ ]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python3 -m spacy download en
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for t in texts:
        doc = nlp(" ".join(t)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
# Do lemmatization keeping only noun, adj, vb, adv
lemma = lemmatization(lemma_docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
selected_prefix = "processing_trained"#@param {type: "string"}
lemma_file_name = selected_prefix + "_" + file_name + ".tsv"

In [ ]:
_lemma = [' '.join(l) for l in lemma]
lemma_pd["lemma"] = _lemma
lemma_pd.to_csv(sep='\t', path_or_buf=analyzed_dir + lemma_file_name)

In [ ]:
lemma_pd = pd.read_csv(analyzed_dir + lemma_file_name, sep="\t")
_lemma = lemma_pd["lemma"]
lemma = [l.split(' ') for l in _lemma]

## Corpus Analysis

In [ ]:
corpus_docs = phrases
corpus_pd = phrases_pd

In [ ]:
# https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
from gensim import corpora

dictionary = corpora.Dictionary(corpus_docs)
rev_dict = {v: k for k, v in dictionary.items()}

In [ ]:
bad_tokens = [
  "covid19", "#covid19", "coronavirus", "#coronavirus", "covid"]
bad_ids = [rev_dict[b] for b in bad_tokens if b in rev_dict]
dictionary.filter_tokens(bad_ids=bad_ids)
dictionary.filter_extremes(no_below=5, no_above=0.05, keep_n=100000, keep_tokens=None)
dictionary.compactify()

In [ ]:
# double check if filters are reasonable
dict_dict = {k: v for k, v in dictionary.items()}
freq_list = [(v, dict_dict[k]) for k, v in dictionary.dfs.items()]
freq_list.sort(reverse=True)
freq_list[:100]

In [ ]:
N = len(dictionary)
corpus = list(dictionary.values())
N

In [ ]:
corpus_filtered = [[e for e in c if e in corpus] for c in corpus_docs]

In [ ]:
selected_prefix = "processing_all"#@param {type: "string"}
corpus_file_name = selected_prefix + "_" + file_name + ".tsv"

In [ ]:
_corpus_filtered = [' '.join(c) for c in corpus_filtered]
corpus_pd["corpus_filtered"] = _corpus_filtered
corpus_pd.to_csv(sep='\t', path_or_buf=analyzed_dir + corpus_file_name)